The csv data comes from a query executed on google cloud https://console.cloud.google.com/bigquery?project=bigqueryncaa&j=bquxjob_2d12840e_1674dc7bb06&page=queryresults  (Improved_Inter_Conference_Games_2015).

The query need to be fixed so that duplicate games are not included in the results. See https://stackoverflow.com/questions/23495734/sql-to-remove-rows-with-duplicated-value-while-keeping-one





In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('InterConf_2015_Dups_Removed.csv')
print("Number of records= ", data.shape[0])
data.head()
data.info()

Number of records=  2216
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2216 entries, 0 to 2215
Data columns (total 14 columns):
game_id            2216 non-null object
season             2216 non-null int64
team_id            2216 non-null object
market             2216 non-null object
name               2216 non-null object
home_team          2216 non-null bool
conf_name          2216 non-null object
opp_conf_name      2216 non-null object
opp_id             2216 non-null object
opp_market         2216 non-null object
opp_name           2216 non-null object
win                2215 non-null object
points_game        2216 non-null int64
opp_points_game    2216 non-null int64
dtypes: bool(1), int64(3), object(10)
memory usage: 227.3+ KB


In [3]:
data.groupby(['conf_name','opp_conf_name']).size()

conf_name              opp_conf_name          
America East           American Athletic           5
                       Atlantic 10                 5
                       Atlantic Coast              7
                       Atlantic Sun                5
                       Big East                    3
                       Big Sky                     1
                       Big South                   2
                       Big Ten                     4
                       Big West                    2
                       Colonial                    4
                       Conference USA              2
                       Horizon                     1
                       Ivy                        16
                       Metro Atlantic Athletic    14
                       Mid Eastern Athletic        2
                       Mid-American                5
                       Mountain West               1
                       Northeast                  17

In [8]:
data[data['win']==True].groupby(['conf_name','opp_conf_name']).agg({
        'win' : ['count']
    })

win
                                          count
conf_name         opp_conf_name                
American Athletic America East                5
Atlantic 10       America East                5
                  American Athletic           6
Atlantic Coast    America East                6
                  American Athletic           7
                  Atlantic 10                14
Atlantic Sun      America East                3
                  American Athletic           1
                  Atlantic 10                 2
Big 12            American Athletic           3
                  Atlantic 10                 4
                  Atlantic Coast              3
                  Atlantic Sun                2
Big East          America East                3
                  American Athletic           6
                  Atlantic 10                 7
                  Atlantic Coast              7
                  Atlantic Sun                2
                  Big 12                      3
Big South         Atlantic Sun                2
                  Big East                    2
Big Ten           America East                4
                  American Athletic           5
                  Atlantic 10                 3
                  Atlantic Coast             11
                  Atlantic Sun                1
                  Big 12                      2
                  Big East                    8
                  Big Sky                     1
                  Big South                   1
...                                         ...
West Coast        Colonial                    1
                  Conference USA              2
                  Horizon                     1
                  Metro Atlantic Athletic     1
                  Mid Eastern Athletic        2
                  Mid-American                2
                  Missouri Valley             1
                  Mountain West               4
                  Northeast                   1
                  Ohio Valley                 3
                  Pacific 12                  4
                  Southeastern                2
                  Southland                   1
                  Southwestern Athletic       3
                  Sun Belt                    1
Western Athletic  American Athletic           1
                  Big Sky                     8
                  Big West                    2
                  Conference USA              4
                  Ivy                         1
                  Mid Eastern Athletic        5
                  Mid-American                1
                  Missouri Valley             1
                  Mountain West               2
                  Northeast                   1
                  Ohio Valley                 4
                  Southeastern                1
                  Southland                   1
                  Southwestern Athletic       6
                  Summit League               4

[289 rows x 1 columns]

In [4]:
conf_summary = data.groupby(['conf_name','opp_conf_name'])
result = conf_summary.size().to_frame(name='counts')
result = result.join(conf_summary.agg({'win' : 'sum'}).rename(columns={'win': 'win_sum'}))
result.reset_index()

,conf_name,opp_conf_name,counts,win_sum
0,America East,American Athletic,5,0
1,America East,Atlantic 10,5,0
2,America East,Atlantic Coast,7,1
3,America East,Atlantic Sun,5,2
4,America East,Big East,3,0
5,America East,Big Sky,1,True
6,America East,Big South,2,2
7,America East,Big Ten,4,0
8,America East,Big West,2,1
9,America East,Colonial,4,1


In [5]:
result['conf_percent'] = result.apply(lambda row: row.win_sum/row.counts * 100, axis=1)


In [64]:
result.head(30)

,conf_name,opp_conf_name,counts,win_sum,conf_percent
0,America East,American Athletic,5,0,0.000000
1,America East,Atlantic 10,5,0,0.000000
2,America East,Atlantic Coast,7,1,14.285714
3,America East,Atlantic Sun,5,2,40.000000
4,America East,Big East,3,0,0.000000
5,America East,Big Sky,1,True,100.000000
6,America East,Big South,2,2,100.000000
7,America East,Big Ten,4,0,0.000000
8,America East,Big West,2,1,50.000000
9,America East,Colonial,4,1,25.000000


In [6]:
result.describe()

,counts,conf_percent
count,432.000000,432.000000
mean,5.129630,57.941507
std,4.600197,38.355036
min,1.000000,0.000000
25%,2.000000,25.000000
50%,4.000000,63.961039
75%,7.000000,100.000000
max,26.000000,100.000000


In [9]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 432 entries, (America East, American Athletic) to (West Coast, Western Athletic)
Data columns (total 3 columns):
counts          432 non-null int64
win_sum         432 non-null object
conf_percent    432 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 11.6+ KB


In [11]:
result = result.reset_index()
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
index            432 non-null int64
conf_name        432 non-null object
opp_conf_name    432 non-null object
counts           432 non-null int64
win_sum          432 non-null object
conf_percent     432 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 20.3+ KB


In [12]:
result.loc[result['conf_name'] == 'Atlantic Coast']

,index,conf_name,opp_conf_name,counts,win_sum,conf_percent
81,81,Atlantic Coast,Atlantic Sun,5,5,100.000000
82,82,Atlantic Coast,Big 12,6,3,50.000000
83,83,Atlantic Coast,Big East,13,6,46.153846
84,84,Atlantic Coast,Big Sky,2,2,100.000000
85,85,Atlantic Coast,Big South,6,6,100.000000
86,86,Atlantic Coast,Big Ten,24,13,54.166667
87,87,Atlantic Coast,Big West,3,2,66.666667
88,88,Atlantic Coast,Colonial,10,7,70.000000
89,89,Atlantic Coast,Conference USA,7,7,100.000000
90,90,Atlantic Coast,Horizon,6,6,100.000000


In [13]:
result.head()

,index,conf_name,opp_conf_name,counts,win_sum,conf_percent
0,0,America East,American Athletic,5,0,0.000000
1,1,America East,Atlantic 10,5,0,0.000000
2,2,America East,Atlantic Coast,7,1,14.285714
3,3,America East,Atlantic Sun,5,2,40.000000
4,4,America East,Big East,3,0,0.000000
